In [ ]:
import requests
import pandas as pd
from pandas import json_normalize 
import json
import flat_table
from mixpanel import Mixpanel
import math

In [ ]:
#this code is to check if we 're able to connect with Mixpanel or not. Respone [200] means mixpanel is connected
requests.get(
  'https://mixpanel.com/api/app/me', 
  auth=([mixpanel_service_account_username], [mixpanel_service_account_password]),
)

<Response [200]>

In [ ]:
#define a function to call GET api from mixpanel in order to get users' profile (including distinct ID)
#for more information, please visit https://developer.mixpanel.com/reference/engage-query
def api_query(page):
    url = "https://eu.mixpanel.com/api/2.0/engage?project_id=[mixpanel_project_id]"
    payload = "output_properties=%5B%22%24DistinctID%22%2C%20%22%24email%22%5D&session_id={i}&page={i}&filter_by_cohort=[mixpanel_cohort_id]&include_all_users=false".format(i=page)
    headers = {
    "accept": "application/json",
    "content-type": "application/x-www-form-urlencoded",
    "authorization": [mixpanel_automated_token]
    }
    response = requests.post(url, data=payload, headers=headers)
    return (response.text)

In [ ]:
#define a function to transfrom json-typed get API response to DataFrame 
def this_page_data(page):
    data = json.loads(api_query(page))
    return pd.json_normalize(data['results'])

In [ ]:
#define a function to see length of get API response length
def length_this_page_data(page):
    data = json.loads(api_query(page))
    this_page_data = pd.json_normalize(data['results'])
    return len(this_page_data)

In [ ]:
#Use While loop to query all corporate users (users who signup with corporate domain emails) and input into a dataframe
#Set df as the Dataframe that contains all users' profile
#The result would be a dataframe with 3 columns: Distinct ID, Email & Last action timestamp
idx = 0
df = pd.DataFrame()
while length_this_page_data(idx) >= 1000:
    idx += 1
    df = df.append(this_page_data(idx))
    print(df)

In [ ]:
#clean data by drop Last action timestamp column and rename Distinct ID, Email columns
df = df.drop(columns=['$properties.$last_seen'])
df = df.rename({'$properties.$email': 'email', '$distinct_id': 'distinctId'}, axis='columns')
df.info()

In [ ]:
#reindex the dataframe
df.reindex()

In [181]:
#drop duplicates
df = df.drop_duplicates()

In [ ]:
#show result dataframe
df

In [271]:
#now user serper.dev API to search each user's email on google and find if any linkedin link in the result
#if yes, extract users' public info such as: full name, company name, title
#the result will be contained in 2 dictionaries: search_link & search_info
url = "https://google.serper.dev/search"

search_link = {}
search_info = {}
for i in range(len(df)):
    payload = json.dumps({
        "q": df.iloc[i,1],
        "gl": "us",
        "hl": "en",
        "autocorrect": True,
        "page": 1
    })
    headers = {
        'X-API-KEY': [your_serper_api_key],
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    data = json.loads(response.text)
    da = json_normalize(data, 'organic')
    for j in range(len(da)):
        if da.iloc[j,1].find('.linkedin.com/') != -1:
            linkedin = da.iloc[j,1]
            info = da.iloc[j,0]
            search_link[df.iloc[i,1]] = linkedin
            search_info[df.iloc[i,1]] = info
            break
        else:
            continue

In [272]:
#transform dict type to dataframe and change column names of dictionary that contains users' emails & linkedin links - dl
dl = pd.DataFrame(list(search_link.items()))
dl = dl.rename(columns = {0:'email', 1:'linkedLink'})

In [273]:
#transform dict type to dataframe and change column names of dictionary that contains users' emails & public info - di
di = pd.DataFrame(list(search_info.items()))
di = di.rename(columns = {0:'email', 1:'publicInfo'})

In [ ]:
#left join df with di and set df_1 to contain its result
df_1 = pd.merge(df, di, how = 'left', on = 'email')

In [276]:
#left join df_1 with dl and set df_2 to contain its result
df_2 = pd.merge(df_1, dl, how = 'left', on = 'email')

In [ ]:
#show df_2
#now we have the dataframe df_2 that contains users' distinct Id, corporate emails, their public info & their linkedin links
df_2

In [279]:
#fill all null values out with blank
df_2 = df_2.fillna('')

In [ ]:
#show df_2 again
df_2

In [281]:
#then we use POST API to send df_2 dataframe back to mixpanel to sync new info collected with current database
for k in range(len(df_2)):
    
    url = "https://api-eu.mixpanel.com/engage?verbose=1#profile-batch-update"
    
    payload = [
        {
            "$token": [mixpanel_project_token],
            "$distinct_id": df_2.iloc[k,0],
            "$set": {"publicInfo": df_2.iloc[k,2]},
            "$set": {"linkedinLink": df_2.iloc[k,3]}
        }
    ]
    headers = {
        "accept": "text/plain",
        "content-type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)

#print response to see if there is any error during sending process
print(response.text)

{"error":null,"status":1}


In [282]:
#export dataframe to a .csv file to for backup
df_2.to_csv(r'file/filter_users_with_public_info.csv',index=False)